This notebook contains the code and all my understandings and everything about magic. This is somehow my first serious work in machine learning.

In [1]:
import torch
import pandas as pd
import numpy as np
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import Dataset, DataLoader
import time

In [2]:
# TODO: make these constants arguments
n_route = 228
n_hist = 12
n_pred = 9
batch_size = 50
epoch = 50
# parser.add_argument('--save', type=int, default=10)
ks = 3  # spatial convolution kernel size
kt = 3  # temporal convolution kernel size
lr = 1e-3
opt = torch.optim.Adam
# parser.add_argument('--graph', type=str, default='default')
inf_mode = 'merge'
rate = 0.1
save = 10
Ko = 4
sch = torch.optim.lr_scheduler.ExponentialLR
decay = 0.1
# Ko changes with n_hist and Kt -> Ko = n_hist - 4 * Kt
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


The model has two spatio temporal blocks and each block has two gated temporal convolutional layers and one spatial convolutional layer in between so we have totaly six layers and we should determine the size of channel for these layers

In [3]:
blocks = [[1, 32, 64], [64, 32, 128]]

We should have a file which defines the graph we want to work on.

In [4]:
W = pd.read_csv("PeMSD7_W_228.csv", header=None)
W_tensor = torch.tensor(W.values)
W_tensor.shape

torch.Size([228, 228])

Equation 10 on original papaer

In [5]:
def weight_matrix(W, sigma2=0.1, epsilon=0.5, scaling=True):
    '''
    Weight matrix function.
    :param W: tensor, the adjacency matrix of the graph.
    :param sigma2: float, scalar of matrix W.
    :param epsilon: float, thresholds to control the sparsity of matrix W.
    :param scaling: bool, whether applies numerical scaling on W.
    :return: np.ndarray, [n_route, n_route].
    '''
    # check whether W is a 0/1 matrix.
    if set(torch.unique(W)) == {0, 1}:
        print('The input graph is a 0/1 matrix; set "scaling" to False.')
        scaling = False

    if scaling:
        n = W.shape[0]
        W = W / 10000. # WHY
        W2, W_mask = W * W, np.ones([n, n]) - np.identity(n)
        # refer to Eq.10
        a = torch.exp(-W2 / sigma2)
        a[a >= epsilon] = 0
        
        return a * W_mask
    
    return W

In [6]:
tmp = weight_matrix(W_tensor)
print(torch.unique(tmp))

tensor([0.0000e+00, 1.4279e-89, 1.0224e-88,  ..., 4.9829e-01, 4.9897e-01,
        4.9999e-01], dtype=torch.float64)


In [7]:
torch.unique(tmp).size()

torch.Size([21397])

In the below cell I want to calculate normalized laplacian.<br/>
I calculate it using \(L=I - D^(-1/2) W D^(-1/2)\)

In [8]:
# Some how different from the researchers code needs more investigation

def scaled_laplacian(W):
    '''
    Scaled graph Laplacian function.
    :param W: np.ndarray, [n_route, n_route], weighted adjacency matrix of G.
    :return: np.matrix, [n_route, n_route].
    '''
    # d ->  diagonal degree matrix
    n, d = np.shape(W)[0], torch.sum(W, axis=1)
    I = torch.eye(n)
    
    # d^(-1/2)
    d = torch.pow(d, -1/2)
    # make d diagonal
    d = torch.diag(d)

    dw = torch.matmul(d, W)
    dwd = torch.matmul(dw, d)
    
    L = I - dwd
    
    # lambda_max \approx 2.0, the largest eigenvalues of L.
    lambdas, _ = torch.eig(L)
    lambdas = lambdas[:, 0]
    lambda_max = torch.max(lambdas)

    return 2 * L / lambda_max - I

In [9]:
z = torch.Tensor([[1, 0, 0], [0, 0, 0], [0, 0, 2]])
x = torch.pow(z, -1/2)
print(x)

tensor([[1.0000,    inf,    inf],
        [   inf,    inf,    inf],
        [   inf,    inf, 0.7071]])


In [10]:
# issue if tmp has zero on the diagonal we'll hae infinity in L and be in trouble
# https://github.com/VeritasYin/STGCN_IJCAI-18/issues/25
# copy the code from the source code
L = scaled_laplacian(tmp)
print(type(L))

<class 'torch.Tensor'>


In [11]:
if True in torch.isinf(L):
    print("sdff")

In [12]:
T = torch.tensor([[0, 2, 3, 4, 6, 7], [2, 0, 5, 8, 9, 10], [3, 5, 0, 11, 12, 13], 
                  [4, 8, 11, 14, 15, 16], [6, 9, 12, 15, 17, 18], [7, 10, 13, 16, 18, 19]], dtype=float)
T = scaled_laplacian(T)
print(T)

tensor([[ 0.7208, -0.1258, -0.1659, -0.1780, -0.2509, -0.2819],
        [-0.1258,  0.7208, -0.2224, -0.2863, -0.3027, -0.3239],
        [-0.1659, -0.2224,  0.7208, -0.3460, -0.3548, -0.3702],
        [-0.1780, -0.2863, -0.3460,  0.3665, -0.3567, -0.3665],
        [-0.2509, -0.3027, -0.3548, -0.3567,  0.3409, -0.3874],
        [-0.2819, -0.3239, -0.3702, -0.3665, -0.3874,  0.3269]],
       dtype=torch.float64)


**Chebyshev polynomials approximation**

In [13]:
# actually this is half of cheb poly
def cheb_poly_approx(L, Ks, n):
    '''
    Chebyshev polynomials approximation function.
    :param L: np.matrix, [n_route, n_route], graph Laplacian.
    :param Ks: int, kernel size of spatial convolution.
    :param n: int, number of routes / size of graph.
    :return: np.ndarray, [n_route, Ks*n_route].
    '''
    L0, L1 = torch.eye(n), L.detach().clone()

    if Ks > 1:
        L_list = [L0.detach().clone(), L1.detach().clone()]
        for i in range(Ks - 2):
            Ln = 2 * L * L1 - L0
            L_list.append(Ln.detach().clone())
            L0, L1 = L1.detach().clone(), Ln.detach().clone()
        # L_lsit [Ks, n*n], Lk [n, Ks*n]
        return torch.cat(L_list, axis=-1)
    elif Ks == 1:
        return L0
    else:
        raise ValueError(f'ERROR: the size of spatial kernel must be greater than 1, but received "{Ks}".')

In [14]:
# Alternative approximation method: 1st approx - first_approx(W, n).
Lk = cheb_poly_approx(L, ks, n_route)
Lk = Lk.to(device)
Lk = Lk.type(torch.cuda.FloatTensor)
# Lk = Lk.type(torch.FloatTensor)

In [15]:
import pandas as pd

data_file = pd.read_csv("PeMSD7_V_228.csv", header=None)
data_file_tensor = torch.tensor(data_file.values)

In [16]:
n_train, n_val, n_test = 34, 5, 5

In [17]:
def seq_gen(len_seq, data_seq, offset, n_frame, n_route, day_slot, C_0=1):
    '''
    Generate data in the form of standard sequence unit.
    :param len_seq: int, the length of target date sequence.  is the number of days
    :param data_seq: np.ndarray, source data / time-series.
    :param offset:  int, the starting index of different dataset type.
    :param n_frame: int, the number of frame within a standard sequence unit,
                         which contains n_his = 12 and n_pred = 9 (3 /15 min, 6 /30 min & 9 /45 min).
    :param n_route: int, the number of routes in the graph.
    :param day_slot: int, the number of time slots per day, controlled by the time window (5 min as default).
    :param C_0: int, the size of input channel.
    :return: np.ndarray, [len_seq, , C_0, n_frame, n_route].
    '''
    # in this example time step is 5 min so a day which is 24 hours will be 288 time steps (day_slot=288) 
    # and n_frame is 21 so we will have 288 - 21 + 1 = 268 (n_slot=268)
    n_slot = day_slot - n_frame + 1

    tmp_seq = torch.zeros((len_seq * n_slot, C_0, n_frame, n_route))
    for i in range(len_seq):
        for j in range(n_slot):
            sta = (i + offset) * day_slot + j
            end = sta + n_frame
            tmp_seq[i * n_slot + j, :, :, :] = torch.reshape(data_seq[sta:end, :], [C_0, n_frame, n_route])
    return tmp_seq

In [18]:
def z_score(x, mean, std):
    '''
    Z-score normalization function: $z = (X - \mu) / \sigma $,
    where z is the z-score, X is the value of the element,
    $\mu$ is the population mean, and $\sigma$ is the standard deviation.
    :param x: np.ndarray, input array to be normalized.
    :param mean: float, the value of mean.
    :param std: float, the value of standard deviation.
    :return: np.ndarray, z-score normalized array.
    '''
    return (x - mean) / std

In [19]:
class Dataset(object):
    def __init__(self, data, stats):
        self.__data = data
        self.mean = stats['mean']
        self.std = stats['std']

    def get_data(self, type):
        return self.__data[type]

    def get_stats(self):
        return {'mean': self.mean, 'std': self.std}

    def get_len(self, type):
        return len(self.__data[type])

    def z_inverse(self, type):
        return self.__data[type] * self.std + self.mean

In [20]:
def data_gen(data_seq, data_config, n_route, n_frame=21, day_slot=288):
    '''
    Source file load and dataset generation.
    :param file_path: str, the file path of data source.
    :param data_config: tuple, the configs of dataset in train, validation, test.
    :param n_route: int, the number of routes in the graph.
    :param n_frame: int, the number of frame within a standard sequence unit,
                         which contains n_his = 12 and n_pred = 9 (3 /15 min, 6 /30 min & 9 /45 min).
    :param day_slot: int, the number of time slots per day, controlled by the time window (5 min as default).
    :return: dict, dataset that contains training, validation and test with stats.
    '''
    n_train, n_val, n_test = data_config
    # generate training, validation and test data
#     data_seq = data.values
    
    seq_train = seq_gen(n_train, data_seq, 0, n_frame, n_route, day_slot)
    seq_val = seq_gen(n_val, data_seq, n_train, n_frame, n_route, day_slot)
    seq_test = seq_gen(n_test, data_seq, n_train + n_val, n_frame, n_route, day_slot)

    # x_stats: dict, the stats for the train dataset, including the value of mean and standard deviation.
    x_stats = {'mean': torch.mean(seq_train), 'std': torch.std(seq_train)}
    print(x_stats)

    # x_train, x_val, x_test: np.array, [sample_size, n_frame, n_route, channel_size].
    x_train = z_score(seq_train, x_stats['mean'], x_stats['std'])
    x_val = z_score(seq_val, x_stats['mean'], x_stats['std'])
    x_test = z_score(seq_test, x_stats['mean'], x_stats['std'])
    
    x_train = x_train.to(device)
    
    print("++++++++")
    print(x_train.type())

    x_data = {'train': x_train, 'val': x_val, 'test': x_test}
    dataset = Dataset(x_data, x_stats)
    return dataset

In [21]:
print(data_file_tensor.type())
PeMS = data_gen(data_file_tensor, (n_train, n_val, n_test), n_route, n_hist + n_pred)

torch.DoubleTensor
{'mean': tensor(58.4998), 'std': tensor(13.7286)}
++++++++
torch.cuda.FloatTensor


In [22]:
# problem

def layer_norm(x):
    '''
    Layer normalization function.
    :param x: tensor, [batch_size, channel, time_step, n_route].
    :return: tensor, [batch_size, channel, time_step, n_route].
    '''
    _, C, _, N = x.size()
    mu, sigma = torch.mean(x, dim=[2, 3], keepdim=True), torch.var(x, dim=[2, 3], keepdim=True)

    gamma = torch.ones([1, C, 1, N])
    beta = torch.zeros([1, C, 1, N])
    # big problem
    _x = (x - mu) / torch.sqrt(sigma + 1e-6) # * gamma + beta
    
    return _x

In [23]:
test = torch.Tensor([[[[1, 2, 3], [4, 5, 6]], [[7, 8, 9], [10, 11, 12]]], [[[13, 14, 15], [16, 17, 18]], 
                                                                           [[19, 20, 21], [22, 23, 24]]]])

print(test.size())
a = torch.nn.Conv2d(2, 1, (1, 1), stride=[1, 1])(test)
print(a)
print(a.size())
# print(layer_norm(test))

torch.Size([2, 2, 2, 3])
tensor([[[[1.3987, 1.5512, 1.7037],
          [1.8561, 2.0086, 2.1611]]],


        [[[3.2284, 3.3809, 3.5334],
          [3.6858, 3.8383, 3.9908]]]], grad_fn=<ThnnConv2DBackward>)
torch.Size([2, 1, 2, 3])


In [24]:
class ST_Network(torch.nn.Module):
    def __init__(self, n_his, Ks, Kt, rate, Lk, Ko):
        '''
        Build the base model.
        :param n_his: int, size of historical records for training.
        :param Ks: int, kernel size of spatial convolution.
        :param Kt: int, kernel size of temporal convolution.
        :param blocks: list, channel configs of st_conv blocks.
        :param rate: the rate of dropout.
        '''
        super(ST_Network, self).__init__()
        self.n_his = n_his
        self.Ks = Ks
        self.Kt = Kt
        self.rate = rate
        self.Lk = Lk
        self.Ko = Ko
        
        print("self.Lk")
        print(self.Lk.type())
        
        # blocks = [1, 32, 64], [64, 32, 128]
        # take a look at the link below for quick reminder of padding, stride and dilation
        # https://github.com/vdumoulin/conv_arithmetic/blob/master/README.md 
        # stride=[1, 1] means no stride
        # padding=[0, 0] means no padding or "Valid"
        # dilation=[1, 1] means no dilation
        self.Conv2DBlock1Temporal1GLU = torch.nn.Conv2d(1, 2 * 32, (Kt, 1), stride=[1, 1], padding=[0, 0], dilation=[1, 1])
        self.SigmoidBlock1Temporal1 = torch.nn.Sigmoid()
        
        self.ThetaBlock1Spatio = torch.nn.Parameter(torch.rand((self.Ks * 32, 32), requires_grad=True)) # problem device and dtype
        self.ReLUBlock1Spatio = torch.nn.ReLU()
        
        self.Conv2DBlock1Temporal2ReLU = torch.nn.Conv2d(32, 64, (Kt, 1), stride=[1, 1], padding=[0, 0], dilation=[1, 1])
        self.ReLUBlock1Temporal2 = torch.nn.ReLU()
        
        self.DropoutBlock1 = torch.nn.Dropout(p=rate)
        
        # problem
        # the size of kernel is one so SAME padding is meaningless
        self.Conv2DBlock2Temporal1 = torch.nn.Conv2d(64, 32, (1, 1), stride=[1, 1])
        self.Conv2DBlock2Temporal1GLU = torch.nn.Conv2d(64, 2 * 32, (Kt, 1), stride=[1, 1], padding=[0, 0], dilation=[1, 1])
        self.SigmoidBlock2Temporal1 = torch.nn.Sigmoid()
        
        self.ThetaBlock2Spatio = torch.nn.Parameter(torch.rand((self.Ks * 32, 32), requires_grad=True)) # problem device and dtype
        self.ReLUBlock2Spatio = torch.nn.ReLU()
        
        self.Conv2DBlock2Temporal2ReLU = torch.nn.Conv2d(32, 128, (Kt, 1), stride=[1, 1], padding=[0, 0], dilation=[1, 1])
        self.ReLUBlock2Temporal2 = torch.nn.ReLU()
        
        self.DropoutBlock2 = torch.nn.Dropout(p=rate)
        
        self.Conv2DOutputTemporal1GLU = torch.nn.Conv2d(128, 2 * 128, (Ko, 1), stride=[1, 1], padding=[0, 0], dilation=[1, 1])
        self.SigmoidOutputTemporal1 = torch.nn.Sigmoid()
        self.Conv2DOutputTemporal2Sigmoid = torch.nn.Conv2d(128, 128, (1, 1), stride=[1, 1], padding=[0, 0], dilation=[1, 1])
        self.SigmoidOutputTemporal2 = torch.nn.Sigmoid()
        # problem
        # the size of kernel is one so SAME padding is meaningless
        self.FullyConnectedLayer = torch.nn.Conv2d(128, 1, (1, 1), stride=[1, 1], dilation=[1, 1])
        
    def forward(self, x):
        # Ko>0: kernel size of temporal convolution in the output layer.
        Ko = self.n_his
        
#         print("forward")
#         print(x.type())
        
        # ST-Block
        # [1, 32, 64]
        x = self.st_conv_block1(x)
        
        # [64, 32, 128]
        x = self.st_conv_block2(x)         

        # Output Layer
        y = self.output_layer(x)

        #     tf.add_to_collection(name='copy_loss',
        #                          value=tf.nn.l2_loss(y - inputs[:, n_his:n_his + 1, :, :]))
        loss_fn = torch.nn.MSELoss()

#         train_loss = loss_fn(y, inputs[:, :, n_his:n_his + 1, :])
#         single_pred = y[:, 0, :, :]
        #     tf.add_to_collection(name='y_pred', value=single_pred)
#         return train_loss, single_pred
        return y

    def st_conv_block1(self, x):
        '''
        Spatio-temporal convolutional block, which contains two temporal gated convolution layers
        and one spatial graph convolution layer in the middle.
        :param x: tensor, batch_size, time_step, n_route, c_in].
        :param Ks: int, kernel size of spatial convolution.
        :param Kt: int, kernel size of temporal convolution.
        :param channels: list, channel configs of a single st_conv block.
        :param scope: str, variable scope.
        :param keep_prob: placeholder, prob of dropout.
        :param act_func: str, activation function.
        :return: tensor, [batch_size, time_step, n_route, c_out].
        '''
        
        x_s = self.block1Temporal1(x)
        x_t = self.block1Spatio(x_s)
        x_o = self.block1Temporal2(x_t)
        x_ln = layer_norm(x_o)
    
        return self.DropoutBlock1(x_ln)
    
    def block1Temporal1(self, x):
        '''
        Temporal convolution layer.
        :param x: tensor, [batch_size, c_in, time_step, n_route].
        :param Kt: int, kernel size of temporal convolution.
        :param c_in: int, size of input channel.
        :param c_out: int, size of output channel.
        :param act_func: str, activation function.
        :return: tensor, [batch_size, c_out, time_step-Kt+1, n_route].
        '''

        _, _, T, n = x.size()

        # if the size of input channel is less than the output,
        # padding x to the same size of output channel.
        # Note, _.get_shape() cannot convert a partially known TensorShape to a Tensor.
        x_input = torch.cat([x, torch.zeros([x.size()[0], 32 - 1, T, n]).to(device)], axis=1)
        
        # keep the original input for residual connection.
        # this is a bad code. We want to use this for residual connections and in order to do 
        # that the dimentions should match so we use "self.Kt - 1:T" just to fix this problem 
        # but we are actually just deleting the first time steps which actually makes a little 
        # sense that we are careing about nearer time steps 
        x_input = x_input[:, :, self.Kt - 1:T, :]
        
        # gated liner unit
    #   tf.add_to_collection(name='weight_decay', value=tf.nn.l2_loss(wt))
    #   is it really the same as padding= 'valid'
        x_conv = self.Conv2DBlock1Temporal1GLU(x)
        
        p = (x_conv[:, 0:32, :, :] + x_input)
        q = x_conv[:, -32:, :, :]

        return p * self.SigmoidBlock1Temporal1(q)
    
    def block1Spatio(self, x):
        '''
        Spatial graph convolution layer.
        :param x: tensor, [batch_size, c_in, time_step, n_route].
        :param Ks: int, kernel size of spatial convolution.
        :param c_in: int, size of input channel.
        :param c_out: int, size of output channel.
        :return: tensor, [batch_size, c_out, time_step, n_route].
        '''
        _, _, T, n = x.size()
        
        x_input = x

    #     variable_summaries(ws, 'theta')
        bs = torch.zeros((32))
        # x -> [batch_size, c_in, time_step, n_route] -> [batch_size, time_step, c_in, n_route]
        # x -> [batch_size, time_step, c_in, n_route] -> [batch_size*time_step, c_in, n_route]
        # x -> [batch_size*time_step, c_in, n_route] -> [batch_size*time_step, c_out, n_route]
        a = torch.reshape(torch.transpose(x, 1, 2), [-1, 32, n])
        x_gconv = self.gconv(a, self.ThetaBlock1Spatio, self.Ks, 32, 32, self.Lk)# + bs
        # [batch_size*time_step, c_out, n_route] -> [batch_size, time_step, c_out, n_route]
        # [batch_size, time_step, c_out, n_route] -> [batch_size, c_out, time_step, n_route]
        x_gc = torch.transpose(torch.reshape(x_gconv, [-1, T, 32, n]), 1, 2)
        
        return self.ReLUBlock1Spatio(x_gc[:, :, :, :] + x_input)
    
    def block1Temporal2(self, x):
        '''
        Temporal convolution layer.
        :param x: tensor, [batch_size, c_in, time_step, n_route].
        :param Kt: int, kernel size of temporal convolution.
        :param c_in: int, size of input channel.
        :param c_out: int, size of output channel.
        :param act_func: str, activation function.
        :return: tensor, [batch_size, c_out, time_step-Kt+1, n_route].
        '''

        _, _, T, n = x.size()

        # if the size of input channel is less than the output,
        # padding x to the same size of output channel.
        # Note, _.get_shape() cannot convert a partially known TensorShape to a Tensor.
        x_input = torch.cat([x, torch.zeros([x.size()[0], 64 - 32, T, n]).to(device)], axis=1)
        
        # keep the original input for residual connection.
        x_input = x_input[:, :, self.Kt - 1:T, :]
        
        x_conv = self.Conv2DBlock1Temporal2ReLU(x)
        
        return self.ReLUBlock1Temporal2(x_conv + x_input)

    def st_conv_block2(self, x):
        '''
        Spatio-temporal convolutional block, which contains two temporal gated convolution layers
        and one spatial graph convolution layer in the middle.
        :param x: tensor, batch_size, time_step, n_route, c_in].
        :param Ks: int, kernel size of spatial convolution.
        :param Kt: int, kernel size of temporal convolution.
        :param channels: list, channel configs of a single st_conv block.
        :param scope: str, variable scope.
        :param keep_prob: placeholder, prob of dropout.
        :param act_func: str, activation function.
        :return: tensor, [batch_size, time_step, n_route, c_out].
        '''     
        x_s = self.block2Temporal1(x)
        x_t = self.block2Spatio(x_s)
        x_o = self.block2Temporal2(x_t)
        x_ln = layer_norm(x_o)
    
        return self.DropoutBlock2(x_ln)
    
    def block2Temporal1(self, x):
        '''
        Temporal convolution layer.
        :param x: tensor, [batch_size, c_in, time_step, n_route].
        :param Kt: int, kernel size of temporal convolution.
        :param c_in: int, size of input channel.
        :param c_out: int, size of output channel.
        :param act_func: str, activation function.
        :return: tensor, [batch_size, c_out, time_step-Kt+1, n_route].
        '''
        _, _, T, n = x.size()

#       PyTorch does not support same padding the way Keras does
#       padding='SAME'
        x_input = self.Conv2DBlock2Temporal1(x)

        
        # keep the original input for residual connection.
        x_input = x_input[:, :, self.Kt - 1:T, :]
        
        # gated liner unit
    #   tf.add_to_collection(name='weight_decay', value=tf.nn.l2_loss(wt))
    #   is it really the same as padding= 'valid'
        x_conv = self.Conv2DBlock2Temporal1GLU(x)
        
        p = (x_conv[:, 0:32, :, :] + x_input)
        q = x_conv[:, -32:, :, :]

        return p * self.SigmoidBlock2Temporal1(q)  

    def block2Spatio(self, x):
        '''
        Spatial graph convolution layer.
        :param x: tensor, [batch_size, c_in, time_step, n_route].
        :param Ks: int, kernel size of spatial convolution.
        :param c_in: int, size of input channel.
        :param c_out: int, size of output channel.
        :return: tensor, [batch_size, c_out, time_step, n_route].
        '''
        _, _, T, n = x.size()
        
        x_input = x

    #     variable_summaries(ws, 'theta')
        bs = torch.zeros((32))
        # x -> [batch_size, c_in, time_step, n_route] -> [batch_size, time_step, c_in, n_route]
        # x -> [batch_size, time_step, c_in, n_route] -> [batch_size*time_step, c_in, n_route]
        # x -> [batch_size*time_step, c_in, n_route] -> [batch_size*time_step, c_out, n_route]
        x_gconv = self.gconv(torch.reshape(torch.transpose(x, 1, 2), [-1, 32, n]), self.ThetaBlock2Spatio, 
                             self.Ks, 32, 32, self.Lk)# + bs problem
        # [batch_size*time_step, c_out, n_route] -> [batch_size, time_step, c_out, n_route]
        # [batch_size, time_step, c_out, n_route] -> [batch_size, c_out, time_step, n_route]
        x_gc = torch.transpose(torch.reshape(x_gconv, [-1, T, 32, n]), 1, 2)
        
        return self.ReLUBlock2Spatio(x_gc[:, :, :, :] + x_input)
        
    def block2Temporal2(self, x):
        '''
        Temporal convolution layer.
        :param x: tensor, [batch_size, c_in, time_step, n_route].
        :param Kt: int, kernel size of temporal convolution.
        :param c_in: int, size of input channel.
        :param c_out: int, size of output channel.
        :param act_func: str, activation function.
        :return: tensor, [batch_size, c_out, time_step-Kt+1, n_route].
        '''   
        _, _, T, n = x.size()

        # if the size of input channel is less than the output,
        # padding x to the same size of output channel.
        # Note, _.get_shape() cannot convert a partially known TensorShape to a Tensor.
        x_input = torch.cat([x, torch.zeros([x.size()[0], 128 - 32, T, n]).to(device)], axis=1)
        
        # keep the original input for residual connection.
        x_input = x_input[:, :, self.Kt - 1:T, :]
        
        x_conv = self.Conv2DBlock2Temporal2ReLU(x)

        return self.ReLUBlock2Temporal2(x_conv + x_input)

    def output_layer(self, x):
        '''
        Output layer: temporal convolution layers attach with one fully connected layer,
        which map outputs of the last st_conv block to a single-step prediction.
        :param x: tensor, [batch_size, channel, time_step, n_route].
        :param T: int, kernel size of temporal convolution.
        :param scope: str, variable scope.
        :param act_func: str, activation function.
        :return: tensor, [batch_size, 1, n_route, 1].
        '''
        # maps multi-steps to one.
        x_i = self.output_layer_temp1_GLU(x)
        x_ln = layer_norm(x_i)
        x_o = self.output_layer_temp2_sigmoid(x_ln)            
            
        # maps multi-channels to one.
        x_fc = self.fully_con_layer(x_o)
        
        return x_fc
    
    def output_layer_temp1_GLU(self, x):
        '''
        Temporal convolution layer.
        :param x: tensor, [batch_size, c_in, time_step, n_route].
        :param Kt: int, kernel size of temporal convolution.
        :param c_in: int, size of input channel.
        :param c_out: int, size of output channel.
        :param act_func: str, activation function.
        :return: tensor, [batch_size, c_out, time_step-Kt+1, n_route].
        '''
        _, _, T, n = x.size()
        
        x_input = x
        
        # keep the original input for residual connection.
        x_input = x_input[:, :, self.Ko - 1:T, :]
        
        # gated liner unit
    #   tf.add_to_collection(name='weight_decay', value=tf.nn.l2_loss(wt))
    #   is it really the same as padding= 'valid'
        x_conv = self.Conv2DOutputTemporal1GLU(x)
        
        p = (x_conv[:, 0:128, :, :] + x_input)
        q = x_conv[:, -128:, :, :]

        return p * self.SigmoidOutputTemporal1(q)
    
    def output_layer_temp2_sigmoid(self, x):
        '''
        Temporal convolution layer.
        :param x: tensor, [batch_size, c_in, time_step, n_route].
        :param Kt: int, kernel size of temporal convolution.
        :param c_in: int, size of input channel.
        :param c_out: int, size of output channel.
        :param act_func: str, activation function.
        :return: tensor, [batch_size, c_out, time_step-Kt+1, n_route].
        '''
        _, _, T, n = x.size()
        
        x_conv = self.Conv2DOutputTemporal2Sigmoid(x)
        
        return self.SigmoidOutputTemporal2(x_conv)

    def fully_con_layer(self, x):
        '''
        Fully connected layer: maps multi-channels to one.
        :param x: tensor, [batch_size, channel, 1, n_route].
        :param n: int, number of route / size of graph.
        :param channel: channel size of input x.
        :return: tensor, [batch_size, 1, n_route, 1].
        '''
    #     tf.add_to_collection(name='weight_decay', value=tf.nn.l2_loss(w))
    #     PyTorch does not support same padding the way Keras does
    #     padding='SAME'
        return self.FullyConnectedLayer(x)
    
    # problem
    def gconv(self, x, theta, Ks, c_in, c_out, kernel):
        '''
        Spectral-based graph convolution function.
        :param x: tensor, [batch_size, c_in, n_route].
        :param theta: tensor, [Ks*c_in, c_out], trainable kernel parameters.
        :param Ks: int, kernel size of graph convolution.
        :param c_in: int, size of input channel.
        :param c_out: int, size of output channel.
        :return: tensor, [batch_size, n_route, c_out].
        '''
        # graph kernel: tensor, [n_route, Ks*n_route]
        n = kernel.shape[0]
        # x -> [batch_size, c_in, n_route] -> [batch_size*c_in, n_route]
        x_tmp = torch.reshape(x, [-1, n])
        # x_mul = x_tmp * ker -> [batch_size*c_in, Ks*n_route] -> [batch_size, c_in, Ks, n_route]
#         print("x_tmp")
#         print(x_tmp.type())
#         print("kernel")
#         print(kernel.type())
        x_mul = torch.reshape(torch.matmul(x_tmp, kernel), [-1, c_in, Ks, n]) # problem should kernel's 
        # requires_grad be true and be on GPU
        # [batch_size, c_in, Ks, n_route] -> [batch_size, n_route, Ks, c_in]
        # [batch_size, n_route, Ks, c_in] -> [batch_size, n_route, c_in, Ks]
        # x_ker -> [batch_size, n_route, c_in, K_s] -> [batch_size*n_route, c_in*Ks]
        x_ker = torch.reshape(torch.transpose(torch.transpose(x_mul, 1, 3), 2, 3), [-1, c_in * Ks])
        # x_gconv -> [batch_size*n_route, c_out] -> [batch_size, n_route, c_out]
#         print("x_ker")
#         print(x_ker.type())
#         print("theta")
#         print(theta.type())
        x_gconv = torch.reshape(torch.matmul(x_ker, theta), [-1, n, c_out])
        # [batch_size, n_route, c_out] -> [batch_size, c_out, n_route]
        x_gconv = torch.transpose(x_gconv, 1, 2) # I added this line, maybe we can also reshape to [-1, c_out, n] in 
        # the previous line instead
        return x_gconv

In [25]:
model = ST_Network(n_hist, ks, kt, rate, Lk, Ko).to(device)

self.Lk
torch.cuda.FloatTensor


In [39]:
def make_train_step(model, loss_fn, optimizer):
    def train_step(x, y):
        # put model in training mode 
        # some models may use mechanisms like Dropout, for instance, which have distinct behaviors in 
        # training and evaluation phase
        model.train()
        
        # compute the model output
        yhat = model(x)
        
        # calculate loss
#         print(type(yhat))
#         print(yhat.size())
#         print("x_batch[:, :, n_hist-2:, :]")
#         print(x_batch[:, :, n_hist-2:, :].size())
        # todo the dimention of yhat is wrong. its dimention 2 value should be 9 not 10
        loss = loss_fn(y, yhat)
#         print(loss)
        
        # compute gradients
        loss.backward()
    
        # update model weights
        optimizer.step()  
        # clear the gradients
        optimizer.zero_grad()
        
        return loss.item()  # the .item() is so important or we'll have problem with memory
    
    # Returns the function that will be calculated inside the train loop
    return train_step


In [40]:
def train(inputs, model, epoch, n_hist):
    # Define model loss
    criterion = torch.nn.MSELoss()
    # define the optimization
    # torch.optim.Adam(lr, weight_decay=0.01).minimize(train_loss)
    optimizer = opt(model.parameters(), lr=lr)
    schedular = sch(optimizer, gamma=decay)
    
    train_step = make_train_step(model, criterion, optimizer)
    losses = []
    
    # enumerate epochs     
    for i in range(epoch):
        start_time = time.time()
        data_loader = DataLoader(inputs.get_data('train'),
                                 batch_size=batch_size, # where did I define batch size
                                 drop_last=True)
        # gen_batch(inputs.get_data('train'), batch_size, dynamic_batch=True, shuffle=True)
        # enumerate mini batches
        for _, x_batch in enumerate(data_loader):
            # x dimention = [batch_size, channel = 1, n_frame, n_route]
            # n_hist = 12
            x = x_batch[:, :, 0:n_hist, :]
            
            # Performs one train step and returns the corresponding loss
            print("y")
            print(x_batch[:, :, n_hist:n_hist+1, :].size())
            loss = train_step(x, x_batch[:, :, n_hist, :])
            losses.append(loss)
        
#         print(model.state_dict())
        print(f'Epoch {i:2d} Training Time {time.time() - start_time:.3f}s')
    
        schedular.step()

#         start_time = time.time()
#         min_va_val, min_val = \
#                 model_inference(sess, pred, inputs, batch_size, n_his, n_pred, step_idx, min_va_val, min_val)

            
#         loss = loss_fn(predictions, t)
#       train_op.step()
#         for ix in tmp_idx:
#             va, te = min_va_val[ix - 2:ix + 1], min_val[ix - 2:ix + 1]
#             print(f'Time Step {ix + 1}: '
#                     f'MAPE {va[0]:7.3%}, {te[0]:7.3%}; '
#                     f'MAE  {va[1]:4.3f}, {te[1]:4.3f}; '
#                     f'RMSE {va[2]:6.3f}, {te[2]:6.3f}.')
#         print(f'Epoch {i:2d} Inference Time {time.time() - start_time:.3f}s')

#         if (i + 1) % save == 0 || (i + 1) == epoch:
        torch.save(model.state_dict(), "model.pt")

In [41]:
train(PeMS, model, epoch, n_hist)
# todo the problem was model.parameter

y
torch.Size([50, 1, 1, 228])
y
torch.Size([50, 1, 1, 228])
y
torch.Size([50, 1, 1, 228])
y
torch.Size([50, 1, 1, 228])
y
torch.Size([50, 1, 1, 228])
y
torch.Size([50, 1, 1, 228])
y
torch.Size([50, 1, 1, 228])
y
torch.Size([50, 1, 1, 228])
y
torch.Size([50, 1, 1, 228])
y
torch.Size([50, 1, 1, 228])
y
torch.Size([50, 1, 1, 228])
y
torch.Size([50, 1, 1, 228])
y
torch.Size([50, 1, 1, 228])
y
torch.Size([50, 1, 1, 228])
y
torch.Size([50, 1, 1, 228])
y
torch.Size([50, 1, 1, 228])
y
torch.Size([50, 1, 1, 228])
y
torch.Size([50, 1, 1, 228])
y
torch.Size([50, 1, 1, 228])
y
torch.Size([50, 1, 1, 228])
y
torch.Size([50, 1, 1, 228])
y
torch.Size([50, 1, 1, 228])
y
torch.Size([50, 1, 1, 228])
y
torch.Size([50, 1, 1, 228])
y
torch.Size([50, 1, 1, 228])
y
torch.Size([50, 1, 1, 228])
y
torch.Size([50, 1, 1, 228])
y
torch.Size([50, 1, 1, 228])
y
torch.Size([50, 1, 1, 228])
y
torch.Size([50, 1, 1, 228])
y
torch.Size([50, 1, 1, 228])


KeyboardInterrupt: 

In [ ]:
# import subprocess
# import sys

# subprocess.check_call([sys.executable, "-m", "pip", "install", "tensorflow"])

In [ ]:
# def multi_pred(model, y_pred, seq, batch_size, n_his, n_pred, step_idx, dynamic_batch=True):
#     '''
#     Multi_prediction function.
#     :param y_pred: placeholder.
#     :param seq: np.ndarray, [len_seq, n_frame, n_route, C_0].
#     :param batch_size: int, the size of batch.
#     :param n_his: int, size of historical records for training.
#     :param n_pred: int, the length of prediction.
#     :param step_idx: int or list, index for prediction slice.
#     :param dynamic_batch: bool, whether changes the batch size in the last one if its length is less than the default.
#     :return y_ : tensor, 'sep' [len_inputs, n_route, 1]; 'merge' [step_idx, len_inputs, n_route, 1].
#             len_ : int, the length of prediction.
#     '''
#     pred_list = []
#     for batch in gen_batch(seq, min(batch_size, len(seq)), dynamic_batch=dynamic_batch):
#         # Note: use np.copy() to avoid the modification of source data.
#         test_seq = torch.copy(batch[:, :, 0:n_his, :])
#         step_list = []
# #         for j in range(n_pred):
#             pred = model(test_seq)
# #             pred = sess.run(y_pred,
# #                             feed_dict={'data_input:0': , 'keep_prob:0': 1.0})
# #             if isinstance(pred, list):
# #                 pred = np.array(pred[0])
#             # move the window to predict the next 9 time steps
# #             test_seq[:, :, 0:n_his - 1, :] = test_seq[:, :, 1:n_his, :]
# #             test_seq[:, :, n_his - 1, :] = pred
#             step_list.append(pred)
#         pred_list.append(step_list)
#     #  pred_array -> [n_pred, batch_size, n_route, C_0)
#     pred_array = np.concatenate(pred_list, axis=1)
#     return pred_array[step_idx], pred_array.shape[1]



In [ ]:
a = [[[[[1], [2]], [[3], [4]]], [[[5], [6]], [[7], [8]]]]]
print(a)
print(np.concatenate(a, axis=1))

In [ ]:
def model_test(inputs, batch_size, n_his, n_pred, inf_mode):
    '''
    Load and test saved model from the checkpoint.
    :param inputs: instance of class Dataset, data source for test.
    :param batch_size: int, the size of batch.
    :param n_his: int, the length of historical records for training.
    :param n_pred: int, the length of prediction.
    :param inf_mode: str, test mode - 'merge / multi-step test' or 'separate / single-step test'.
    :param load_path: str, the path of loaded model.
    '''
    start_time = time.time()

    model = ST_Network(n_hist, ks, kt, rate, Lk, Ko).to(device)
    # the load_state_dict() function takes a dictionary object not a path to a saved object
    model.load_state_dict(torch.load("model.pt"))
        
    # to set dropout and batch normalization layers to evaluation mode before running inference
    model.eval()
    print(f'>> Loading saved model from model.pt ...')

    # don't care about the further times at the moment
#        if inf_mode == 'sep':
#            # for inference mode 'sep', the type of step index is int.
#            step_idx = n_pred - 1
#            tmp_idx = [step_idx]
#        elif inf_mode == 'merge':
#            # for inference mode 'merge', the type of step index is np.ndarray.
#            step_idx = tmp_idx = np.arange(3, n_pred + 1, 3) - 1
#        else:
#            raise ValueError(f'ERROR: test mode "{inf_mode}" is not defined.')

    loss_fn = torch.nn.MSELoss()
    # without making the data into batches we'll have cuda out of memory error
#     for batch in gen_batch(seq, min(batch_size, len(seq)), dynamic_batch=dynamic_batch):
    x_stats = inputs.get_stats()
    data_loader = DataLoader(inputs.get_data('test'),
                                 batch_size=10, # where did I define batch size
                                 drop_last=True)
        # gen_batch(inputs.get_data('train'), batch_size, dynamic_batch=True, shuffle=True)
        # enumerate mini batches
    for _, x_batch in enumerate(data_loader):
        x_batch = x_batch.to(device)
#         y_test, len_test = multi_pred(model, pred, x_test, batch_size, n_his, n_pred, step_idx)
        pred = model(x_batch)
#         evl = evaluation(x_test[0:len_test, step_idx + n_his, :, :], y_test, x_stats)

#         for ix in tmp_idx:
#             te = evl[ix - 2:ix + 1]
#             print(f'Time Step {ix + 1}: MAPE {te[0]:7.3%}; MAE  {te[1]:4.3f}; RMSE {te[2]:6.3f}.')
           
        loss = loss_fn(x_batch[:, :, n_his, :], pred)
        print(loss.item()) #, MAE(v, v_), RMSE(v, v_)])
    
    print(f'Model Test Time {time.time() - start_time:.3f}s')
    print('Testing model finished!')
    

In [ ]:
# model_test(PeMS, PeMS.get_len('test'), n_hist, n_pred, inf_mode)